### **Question E | Multiresolution Correlation, Hurst Exponent and Annualized Volatility**

> 1. With Haar wavelets and the dataset provided with TD5, determine the multiresolution correlation
between all the pairs of FX rates, using GBPEUR, SEKEUR, and CADEUR (work with the average between
the highest and the lowest price and transform this average price in returns on the smallest time step).
Do you observe an Epps effect and how could you explain this?

In [ ]:
# Open original data file
import pandas as pd
raw_data = pd.read_excel("Dataset TD5.xlsx", header=None)
raw_data.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GBPEUR Curncy,NaN,NaN,NaN,SEKEUR Curncy,NaN,NaN,NaN,CADEUR Curncy,NaN,NaN
2,Date,HIGH,LOW,NaN,Date,HIGH,LOW,NaN,Date,HIGH,LOW


In [72]:
# We fix the data format by handpicking columns to make the dataframe we want
df = pd.DataFrame()

# For pound - euro
df["high_GBPEUR"] = raw_data.iloc[3:, 1] # 2nd column starting from 4th row
df["low_GBPEUR"]  = raw_data.iloc[3:, 2] # 3rd column starting from 4th row

# For sweden crown - euro
df["high_SEKEUR"] = raw_data.iloc[3:, 5] # 6th column starting from 4th row
df["low_SEKEUR"]  = raw_data.iloc[3:, 6] # 7th column starting from 4th row

# For canada dollar - euro
df["high_CADEUR"] = raw_data.iloc[3:, 9] # 10th column starting from 4th row
df["low_CADEUR"]  = raw_data.iloc[3:, 10] # 11th column starting from 4th row

# Then we add the date 
df["date"] = raw_data.iloc[3:, 0] # 1st column starting from 4th row
# Fixed data
df

,high_GBPEUR,low_GBPEUR,high_SEKEUR,low_SEKEUR,high_CADEUR,low_CADEUR,date
3,1.2932,1.2917,0.10725,0.1072,0.6842,0.6829,2016-03-07 08:59:59.990000
4,1.294,1.293,0.10728,0.10717,0.6849,0.6841,2016-03-07 09:15:00
5,1.2943,1.2922,0.10726,0.10719,0.6844,0.6837,2016-03-07 09:30:00
6,1.293,1.2913,0.10728,0.10721,0.6844,0.6839,2016-03-07 09:45:00
7,1.2931,1.2921,0.10725,0.10719,0.684,0.6835,2016-03-07 10:00:00
...,...,...,...,...,...,...,...
12927,1.1879,1.1867,0.10536,0.10531,0.6897,0.6893,2016-09-07 17:00:00
12928,1.1883,1.1874,0.10537,0.10534,0.6902,0.6895,2016-09-07 17:15:00
12929,1.188,1.1874,0.10538,0.10536,0.6902,0.6898,2016-09-07 17:30:00
12930,1.1874,1.1866,0.10537,0.10536,0.6902,0.6901,2016-09-07 17:45:00


In [73]:
# First we compute the average between the lowest and the highest price
# We do the operation for each fx rate
df['mid_GBPEUR'] = (df['high_GBPEUR']+df['low_GBPEUR'])/2
df['returns_GBPEUR']=df['mid_GBPEUR'].pct_change()
df['mid_SEKEUR'] = (df['high_SEKEUR']+df['low_SEKEUR'])/2
df['returns_SEKEUR']=df['mid_SEKEUR'].pct_change()
df['mid_CADEUR'] = (df['high_CADEUR']+df['low_CADEUR'])/2
df['returns_CADEUR']=df['mid_CADEUR'].pct_change()
df = df.drop(3, axis=0) # We remove the line with NaNs
df.head()

C:\Users\Adrien\AppData\Local\Temp\ipykernel_17736\4129758546.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['returns_GBPEUR']=df['mid_GBPEUR'].pct_change()
C:\Users\Adrien\AppData\Local\Temp\ipykernel_17736\4129758546.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['returns_SEKEUR']=df['mid_SEKEUR'].pct_change()
C:\Users\Adrien\AppData\Local\Temp\ipykernel_17736\4129758546.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False)

,high_GBPEUR,low_GBPEUR,high_SEKEUR,low_SEKEUR,high_CADEUR,low_CADEUR,date,mid_GBPEUR,returns_GBPEUR,mid_SEKEUR,returns_SEKEUR,mid_CADEUR,returns_CADEUR
4,1.294,1.293,0.10728,0.10717,0.6849,0.6841,2016-03-07 09:15:00,1.2935,0.000812,0.107225,0.000000e+00,0.6845,1.389803e-03
5,1.2943,1.2922,0.10726,0.10719,0.6844,0.6837,2016-03-07 09:30:00,1.29325,-0.000193,0.107225,-1.110223e-16,0.68405,-6.574142e-04
6,1.293,1.2913,0.10728,0.10721,0.6844,0.6839,2016-03-07 09:45:00,1.29215,-0.000851,0.107245,1.865237e-04,0.68415,1.461881e-04
7,1.2931,1.2921,0.10725,0.10719,0.684,0.6835,2016-03-07 10:00:00,1.2926,0.000348,0.10722,-2.331111e-04,0.68375,-5.846671e-04
8,1.2926,1.2921,0.10724,0.10718,0.6839,0.6836,2016-03-07 10:15:00,1.29235,-0.000193,0.10721,-9.326618e-05,0.68375,-1.110223e-16


Haar wavelets are useful to approximate functions and follow the expression:
$$\psi(t) =
\begin{cases}
-1, & 0 \le t < \tfrac{1}{2}, \\
\;\;1, & \tfrac{1}{2} \le t < 1, \\
0, & \text{otherwise}.
\end{cases}$$

The bigger the step, the more we approximate the true shape of the function, it's like a blurry filter.

Originally, we thought the method to execute a Haar wavelet transformation used a sort of rolling window (overlap), but according to the Wikipedia page on wavelet transforms, the classic method for calculating the *Haar* wavelets does not use overlap (https://en.wikipedia.org/wiki/Discrete_wavelet_transform), so we won't use overlap and will "lose" half the data at each time frame increment.

In [ ]:
import numpy as np

def haar_transformation(data):
    # make the returns data into a numpy array
    a = data.values
    coeffs = []
    
    # We loop as long as possible, so until the size of the scale is half the dataset
    while len(a) >= 2:
        # If odd length, drop the last one to ensure pairing
        if len(a) % 2 != 0:
            a = a[:-1]
            
        # We take steps of 2 every time, allowing us to calculate all differences and approximations at once
        # even = a[0], a[2], a[4], etc
        # odd  = a[1], a[3], a[5], etc
        # Here we got two arrays containing the [0, 1/2[ and [1/2, 1[ of each step
        even = a[0::2]
        odd  = a[1::2]
        
        # First we start by storing the difference
        d = (even - odd) / np.sqrt(2)
        coeffs.append(d)
        
        # Second, we pass the sum array to the next level
        # This becomes the input 'a' for the next iteration
        next_a = (even + odd) / np.sqrt(2)
        a = next_a
        
    # To finish we also store the last remaining sum, which represents the overall trend of the series
    coeffs.append(a) 
    
    return coeffs

> 2. Calculate the Hurst exponent of GBPEUR, SEKEUR, and CADEUR. Determine their annualized
volatility using the daily volatility and Hurst exponents.